In [13]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import psoap
from psoap.data import lkca14
from psoap import matrix_functions
from psoap import covariance

from matplotlib.ticker import FormatStrFormatter as FSF
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator

from matplotlib.colors import Normalize

In [85]:
ax_size = 2 # in
left_margin = 0.5 # in
middle_margin = 0.7 # in
right_margin = 0.7 # in
top_margin = 0.8 # in
bottom_margin = 0.48 # in

label_width = 0.12 # in
label_pad = 0.1 # in

cb_width = 0.1 # in
cb_height = 2.4 # in

xx = 2 * ax_size + left_margin + middle_margin + right_margin
yy = top_margin + ax_size + bottom_margin

print("Figure is {} x {}".format(xx, yy))

Figure is 5.9 x 3.28


In [80]:
order = 22
wl0 = 5165
wl1 = 5170

In [81]:
a = 0.26
l = 6.0

In [82]:
lkca14.sort_by_SN()
n_epochs = 3
wl = lkca14.wl[0:n_epochs,order,:]
# Sort by ind
ind = (wl[0] > wl0) & (wl[0] < wl1)

wl = wl[:,ind]
fl = lkca14.fl[0:n_epochs,order,ind]
sigma = lkca14.sigma[0:n_epochs,order,ind]
date1D = lkca14.date1D[0:n_epochs]

In [83]:
N = len(wl.flatten())
V11 = np.empty((N,N), dtype=np.float64)
matrix_functions.fill_V11_f(V11, wl.flatten(), a, l)

VN = 50 * np.diag(sigma.flatten()**2)


In [86]:
fig = plt.figure(figsize=(xx, yy))

ax_F = fig.add_axes([left_margin/xx, bottom_margin/yy, ax_size/xx, ax_size/yy])
ax_N = fig.add_axes([(left_margin + middle_margin + ax_size)/xx, bottom_margin/yy, ax_size/xx, ax_size/yy])

ax_cb = fig.add_axes([(left_margin + middle_margin + 2 * ax_size + 0.4 * (right_margin - cb_width))/xx, 
                       0.5 * (yy - cb_height)/yy, cb_width/xx, cb_height/yy])

# ext = (wl0, wl1, wl0, wl1)
maximum = np.max(np.concatenate((V11.flatten(), VN.flatten())))
print(maximum)
norm = Normalize(vmin=0.0, vmax=maximum)

ax_F.imshow(V11, interpolation="none", cmap="viridis", origin="upper", norm=norm) #, extent=ext)

im = ax_N.imshow(VN, interpolation="none", cmap="viridis", origin="upper", norm=norm)

# ax.xaxis.set_major_locator(MultipleLocator(2.0))
# ax.yaxis.set_major_locator(MultipleLocator(2.0))


# Create an axis that is the pixel index
# Then label (1,1) (2,3) etc. In the caption we can explain what these reference.

de = N/n_epochs

x_offset = 0.75 * de
y_offset = 0.2 * de

lw=0.3

for ax in [ax_F, ax_N]:
    ax.axhline(de, color="w", lw=lw)
    ax.axhline(2 * de, color="w", lw=lw)
    ax.axvline(de, color="w", lw=lw)
    ax.axvline(2 * de, color="w", lw=lw)
    
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])

    for i in range(n_epochs):
        for j in range(n_epochs):
            ax.annotate("({},{})".format(j+1,i+1), (x_offset + i * de, y_offset + j * de), 
                        color="w", size="x-small", ha="center")

# Now set up labels for the wavelengths
# Do this by creating ghost axes

ax_F_label_l = fig.add_axes([(left_margin - label_width - label_pad)/xx, bottom_margin/yy, label_width/xx, ax_size/yy])
ax_F_label_b = fig.add_axes([left_margin/xx, (bottom_margin - label_width - label_pad)/yy, ax_size/xx, label_width/yy])    

ax_N_label_l = fig.add_axes([(left_margin + ax_size + middle_margin - label_width - label_pad)/xx, bottom_margin/yy, label_width/xx, ax_size/yy])
ax_N_label_b = fig.add_axes([(left_margin + ax_size + middle_margin)/xx, (bottom_margin - label_width - label_pad)/yy, ax_size/xx, label_width/yy])    

for (ax_label_l, ax_label_b) in [(ax_F_label_l, ax_F_label_b), (ax_N_label_l, ax_N_label_b)]:

    dlabel = 0.333 / 2

    ax_label_l.axvline(0.5, color="k", lw=0.8)    
    ax_label_b.axhline(0.5, color="k", lw=0.8)    
    
    for val in [0, 0.333, 0.666, 1.0]:
        ax_label_l.axhline(val, color="k", lw=0.8)
        ax_label_b.axvline(val, color="k", lw=0.8)

    ax_label_l.text(0.5, dlabel, r"$\mathbf{\lambda}_3$", ha="center", va="center", backgroundcolor="w")
    ax_label_l.text(0.5, 0.333 + dlabel, r"$\mathbf{\lambda}_2$", ha="center", va="center", backgroundcolor="w")
    ax_label_l.text(0.5, 0.666 + dlabel, r"$\mathbf{\lambda}_1$", ha="center", va="center", backgroundcolor="w")

    ax_label_b.text(dlabel, 0.5, r"$\mathbf{\lambda}_1$", ha="center", va="center", backgroundcolor="w")
    ax_label_b.text(0.333 + dlabel, 0.5, r"$\mathbf{\lambda}_2$", ha="center", va="center", backgroundcolor="w")
    ax_label_b.text(0.666 + dlabel, 0.5, r"$\mathbf{\lambda}_3$", ha="center", va="center", backgroundcolor="w")    

    for ax in [ax_label_l, ax_label_b]:    
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis("off")

    
# Create the labels
plt.figtext((0.5 * left_margin)/xx, (bottom_margin + ax_size + 0.5 * top_margin)/yy, 
            r"$\mathbf{A}$", ha="center", va="center", fontsize=16)

plt.figtext((left_margin + 0.05 * ax_size)/xx, (bottom_margin + ax_size + 0.5 * top_margin)/yy, 
            r"$\mathbf{=}$", ha="center", va="center", fontsize=16)

plt.figtext((left_margin + 0.5 * ax_size)/xx, (bottom_margin + ax_size + 0.5 * top_margin)/yy, 
            r"$\mathbf{\Sigma}_F$", ha="center", va="center", fontsize=16)

plt.figtext((left_margin + ax_size + 0.5 * middle_margin)/xx, (bottom_margin + ax_size + 0.5 * top_margin)/yy, 
            r"$\mathbf{+}$", ha="center", va="center", fontsize=16)

plt.figtext((left_margin + 1.5 * ax_size + middle_margin)/xx, (bottom_margin + ax_size + 0.5 * top_margin)/yy, 
            r"$\mathbf{\Sigma}_N$", ha="center", va="center", fontsize=16)




cbar = fig.colorbar(im, cax=ax_cb)
cbar.ax.tick_params(labelsize=8) 

fig.subplots_adjust(left=0.2, right=0.9)
fig.savefig("V11_multi_epoch.pdf")
fig.savefig("V11_multi_epoch.png")
plt.close("all")

0.0676
